In [1]:
# Imports (takes 4-5s)
import cv2
import numpy as np
import os
import pandas as pd

import matplotlib.pyplot as plt
from skimage.color import rgb2gray

from skimage import measure
from skimage import data, img_as_float
from skimage import exposure
from skimage import filters

from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier

import plotly.express as px

192

In [ ]:
#========= Sources
#CHMI: https://www.chmi.cz/files/portal/docs/meteo/kam/ , last access: 20 February 2023;
#University of Utah: https://home.chpc.utah.edu/~u0553130/Camera_Display/wbbs.html, last access: 20 February 2023;
#DWD: https://opendata.dwd.de/weather/webcam/Offenbach-W/, last access: 20 February 2023;

In [ ]:
def showfcmap(figname,cmap='viridis'):
    # Shows picture in the colormap
    plt.figure(figsize = (20,20))
    plt.imshow(figname,cmap=cmap)
    plt.show()

def showf(figname):
    # Shows figure
    plt.figure(figsize = (20,20))
    plt.imshow(figname)
    plt.show()

def cut_point_CHMI(path_f,u=0,d=100000,l=0,r=100000):
    #preprocesses image and crops to the object defined by u,d,l,r
    im_t = cv2.imread(path_f)
    img = rgb2gray(im_t)

    #Setting the points for cropped image - cuts name, time and text( for CHMI cameras img[180::,0::])
    cropped = img[180::,0::]
    cropped = exposure.rescale_intensity(img)
    gamma_corrected = exposure.adjust_gamma(cropped, 0.6)
    edge_sobel = filters.sobel(gamma_corrected)
    house = edge_sobel[u:d,l:r]
    return img

def show_val_px(path):
    #shows picture in plotly.express - tooltiptext with pixel position
    original = cv2.imread(path)
    fig = px.imshow(original)
    fig.show()

In [ ]:
directory = './IMG'
 
# iterate over files in  that directory
#====== zentralbank 1502,1705,1997,2099
#====== Commerzbank 1549,1699,1935,1985
B = [1549,1699,1935,1985]

# For values where all obsect were not visible these were named 'FG',
# others training were named OK, or ON - for "Ok Night"
# You can use different approach placing training set into different directories,etc.

data = []
labels = []
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    #print(f)
    if 'ON' in filename:
        im = cut_point(f,B)
        labels.append('N')
        data.append(im)
    elif 'FG' in filename:
        im = cut_point(f,B)
        labels.append('FG')
        data.append(im)
    elif 'OK' in filename:
        im = cut_point(f,B)
        labels.append('OK')
        data.append(im)
    elif 'OK3' in filename:
        im = cut_point(f,B)
        labels.append('OK')
        data.append(im)
    #showf(im)

model = KNeighborsClassifier(n_neighbors=1)

# encode the labels as integers
le = preprocessing.LabelEncoder()
labels = le.fit_transform(labels)

labels = np.asarray(labels)
data = np.asarray(data)

# reshape data from 3 to  dims
nsamples, nx, ny = data.shape
d2_train_dataset = data.reshape((nsamples,nx*ny))

#train KNN,
knnc = model.fit(d2_train_dataset, labels)

In [ ]:
#@title Testing predictions

# assign directory
directory = '/content/IMG'
 
# iterate over files in  that directory, skip if they were used in training
# this could be shortened, but it was kept for future use (different filenames, ect)
dataT = []
labels = []
fnames = []
for filename in os.listdir(directory):
    try:
        f = os.path.join(directory, filename)
        print(f)
        if 'FG' in filename:
            continue
        elif 'OK' in filename:
            continue
        elif 'ON' in filename:
            continue
        elif 'OK3' in filename:
            continue
        else:
            im = cut_point(f,B)
        dataT.append(im)
        fnames.append(filename)
      #print(filename)
      #showf(im)
    except Exception as e:
        print(e)
        print(filename)

dataT = np.asarray(dataT)

# reshape data from 3 to  dims
nsamples, nx, ny = dataT.shape
d2_test_dataset = dataT.reshape((nsamples,nx*ny))

vysl = []
for id,z in enumerate(d2_test_dataset):
    pred= knnc.predict(z.reshape(1, -1))
    vysl.append(pred)
    print(pred, fnames[id])

In [ ]:
# Read values from label encoding
predic_decoded = list(le.inverse_transform(vysl))

In [ ]:
#@title Save models
import pickle 

# Define list of models (somethong like knn, clf...) 
# Save it in the loop, if there are multiple of them 

models = [knnc_mtn_tops,knnc_mtn_tops2,knnc_mtn_back,knnc_mountains,knnc_building,knnc_stadium ]
for ind,val in enumerate(models):
    nam = str(ind)
    knnPickle = open(f'/content/knnpickle{nam}_Ffile', 'wb') 
    # source, destination 
    pickle.dump(val, knnPickle)  
    # close the file
    knnPickle.close()